# Linear Regression

### 🏡 Case Study : Predicting House Price

- Kaggle의 주택 판매 가격 데이터를 통해 회귀 문제를 풀어보도록 하겠습니다.
- 이번 노트에서 사용하는 변수는 다음과 같습니다.
    - `SalePrice`: 우리가 예측해야 하는 타겟값으로 주택판매가격(달러) 입니다.
    - `OverallQual`: 전반적인 퀄리티
    - `GrLivArea`: 지상 생활면적(square feet)
<br></br>

데이터에 대한 더 구체적인 설명은 링크를 참고해주세요.
["House Prices: Advanced Regression Techniques"](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview)

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

# 주어진 url 주소를 이용해 house prices 데이터를 가져옵니다.
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/house-prices/house_prices_train.csv')
df_t = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/house-prices/house_prices_test.csv')

In [ ]:
# shape과 head()로 데이터를 살펴봅니다.
print(df.shape)
df.head()

In [ ]:
print(df_t.shape)
df_t.head()

df_t의 컬럼이 df의 컬럼보다 하나 적습니다. 왜 그럴까요?

In [ ]:
## 여러 특성 중 'GrLivArea', 'OverallQual', 'SalePrice'를 사용합니다.

df = df[['GrLivArea', 'OverallQual', 'SalePrice']]
df_t = df_t[['GrLivArea', 'OverallQual']]

#### 지상생활면적(GrLivArea)을 통해 주택 판매 가격(SalesPrice)을 예측해보기로 합시다.  

##### 1. 기존 경험을 통해 예측
- 마침 우리가 살고 있는 집의 크기가 1750sqft이고 주택 판매 가격이 124,000달러라고 해봅시다. 그렇다면 대충 집의 크기가 1700sqft 수준이라면 주택 판매 가격이 120,000달러 대일 것이라고 예측할 수 있을 것입니다.
- 기존 경험을 통한 예측은 간단하지만 오류가 생길 수 있습니다.

In [ ]:
con = (df['GrLivArea']>=1700) & (df['GrLivArea']< 1800)

print('최저 주택 판매 가격:', df.loc[con, 'SalePrice'].min())
print('최고 주택 판매 가격:', df.loc[con, 'SalePrice'].max())

집의 크기가 1700에서 1800 사이인 주택 판매 가격을 확인해봤을 때 기존 경험과 달리 주택 가격의 범위가 굉장히 넓은 것을 알 수 있습니다.

##### 2. 통계 정보 활용
- 통계 정보를 활용하여 집의 가격을 예측할 수 있습니다.
- 집의 크기가 1700에서 1800 사이인 주택의 `SalePrice`의 통계 정보를 확인해 보겠습니다.

In [ ]:
# pandas 포멧팅을 사용하면, 판다스 객체들의 결과에서 쉼표를 넣고 소수점 아래는 생략하도록 설정할 수 있습니다.
# float 형식 소수점 1자리, 쉼표 포함
pd.set_option('display.float_format', '{:,.1f}'.format)

# 기본 세팅으로 돌아오기 위해서는 아래 코드를 실행하면 됩니다.
# pd.set_option('display.float_format', None)

con = (df['GrLivArea']>=1700) & (df['GrLivArea']< 1800)
df.loc[con, 'SalePrice'].describe()

집의 크기가 1700에서 1800 사이인 주택의 SalePrice는 평균적으로 201,988.8 달러임을 알 수 있습니다.  
전체 주택 판매 가격의 통계 정보도 확인해보겠습니다.

In [ ]:
df['SalePrice'].describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## SalePrice의 확률밀도함수를 그려보겠습니다
sns.displot(df['SalePrice'], kde=True)

## 평균과, 중앙값으로 수직선을 그려보겠습니다.
plt.axvline(df['SalePrice'].mean(), color='blue')
plt.axvline(df['SalePrice'].median(), color='red');

- 통계적 방법으로 특정값을 예측하는 것은 일상적으로도 많이 사용되는 방법입니다.
- 중간값이나 평균은 가장 간단하고 직관적인 방법으로 주택 판매 가격을 예측하기 위한 시작점으로 삼기에 좋습니다.

##### 3. 예측 모델 활용
- 주택 판매 가격을 예측하기 위해 머신러닝 알고리즘을 활용하여 예측모델을 만드는 방법이 있습니다.
- 회귀예측 알고리즘 중 가장 단순하고 대표적인 선형회귀에 대해 알아보겠습니다.

**기준모델**
- 예측 모델을 만들기에 앞서 가장 간단하면서도 직관적이며 최소한의 성능을 나타내는 `기준`이 되는 모델이 필요합니다.
    - 앞으로 우리가 만들 예측 모델이 정말 의미있는 모델인지 판단하기 위해서는 하한선이 되는 기준이 있어야합니다.
<br></br>

- 기준모델은 설정하기 쉽고 적절한 결과를 제공할 수 있으며 예측 모델과 비교할 수 있는 기준이 되는 알고리즘이어야합니다.
    - 간단하게는 통계값을 사용한 기준모델을 사용할 수 있습니다.
        - 회귀문제: 타겟의 평균값 (평균 기준모델)
        - 분류문제: 타겟의 최빈 클래스 (최빈값 기준모델)
        - 시계열회귀문제: 이전 타임스탬프의 값
    - 그 외 앞으로 배우게 될 간단한 예측 모델도 기준모델이 될 수 있습니다.
        - 선형 회귀, 로지스틱 회귀 등
<br></br>

- 아래 그래프는 집의 크기와 주택 판매 가격의 관계를 나타낸 산점도입니다.
- 이 그래프를 가장 잘 나타내는 하나의 직선을 단순선형회귀 모델이라고 부릅니다.


In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=df['GrLivArea'], y = df['SalePrice'])
plt.show()

산점도 위에 기준모델을 그려보겠습니다.

In [ ]:
# predict: 평균 기준모델
plt.figure(figsize=(8,6))
predict = df['SalePrice'].mean()
sns.lineplot(x=df['GrLivArea'], y=predict, color='red')

sns.scatterplot(x=df['GrLivArea'], y = df['SalePrice'])
plt.show()

그래프 상으로 봤을 때 평균 기준모델은 그래프에 가장 잘 맞는 직선으로 보이지는 않습니다.  
그렇다면 가장 잘 맞는 직선은 어떻게 찾을 수 있을까요?

## 2.1 선형회귀모델

- 산점도(scatterplot)에 가장 잘 맞는(best fit) 직선을 그려주면 그것이 회귀 예측 모델이 됩니다.
- 가장 잘 맞는 회귀선을 어떻게 구할 수 있을까요?

단순하게 생각해봅시다.  
- 모델의 예측값과 실제값의 차이가 0이라면 모델은 실제값을 정확하게 예측하는 최고의 모델일 것입니다.
- `(실제값) - (모델의 예측값)`을 **잔차(residual)**라고 하며 잔차를 최소로 하는 선을 찾으면 됩니다.  
    - 실제값 : $ y $
    - 단순선형회귀모델(예측값) : ${\hat y} = \hat{\alpha}x + \hat{\beta} $
    - 잔차 : $ y - (\hat{\alpha}x + \hat{\beta}) $  
    cf) 오차(error)는 모집단에서의 예측값과 실제값의 차이를 말합니다. 하지만 통상적으로는 같은 의미로 사용됩니다.
    

<br></br>

최적의 선형회귀선은 잔차 제곱들의 합인 RSS(residual sum of squares)를 최소로 하는 직선입니다.

$$ RSS = \sum_{i=1}^{N} {\hat{e_i}}^2 =  \sum_{i=1}^{N} {(y_i - (\hat{\alpha}x_i + \hat{\beta}))}^2 $$

RSS는 선형회귀모델의 **비용함수**입니다. 모델을 학습한다는 것은 이 비용함수를 최소로 하도록 하는 파라미터 $\alpha$와 $\beta$를 찾는 것입니다.  
$\alpha$, $\beta$는 비용함수를 미분하여 나온 아래와 같은 식을 통해 구할 수 있습니다. 아래 식을 도출하는 과정은 참고문서를 참조해주세요.

$ \hat{\alpha} = \frac{S_{xy}}{S_{xx}} $  

$ \hat{\beta} = \bar{y} - \hat{\alpha}\bar{x}$


$ S_{xy}= {\sum_{i=1}^{N}(x_i-\bar{x})(y_i-\bar{y})}$

$ S_{xx} = {\sum_{i=1}^{N}(x_i-\bar{x})^2}$


이런 식으로 잔차 제곱의 합을 최대한 작게, 즉 **최소**로 만들어 주는 방법을 **최소제곱법**, 영어로는 **Ordinary Least Square**, 줄여서 **OLS**라고 부릅니다.  

<img src=https://imgur.com/4ETZhWe.png width=50%>  

[Ordinary Least-Squares Regression](https://datajobs.com/data-science-repo/OLS-Regression-[GD-Hutcheson].pdf)
<br></br>

In [ ]:
## Seaborn regplot으로 회귀선을 그릴 수 있습니다.
plt.figure(figsize=(8,6))
sns.regplot(x=df['GrLivArea'], y=df['SalePrice']);

선형회귀 직선은 독립변수(independent variable, x)가 변화함에 따라 종속변수(dependent variable, y)가 어떻게 변화하는지 두 변수 간의 관계를 요약해줍니다.

- **종속변수는 반응(Response)변수, 레이블(Label), 타겟(Target)등으로 불립니다.**
- **독립변수는 예측(Predictor)변수, 설명(Explanatory)변수, 특성(feature) 등으로 불립니다.**

---
### scikit-learn을 사용해서 선형회귀모델을 만들어 보겠습니다.
- scikit-learn은 머신러닝 모델을 만드는 데 가장 많이 사용하는 라이브러리로 이번 섹션에서 계속 활용됩니다.

- scikit-learn을 활용해 모델을 만들고 데이터를 분석하기 위해서는 다음과 같은 데이터 구조를 사용해야 합니다:

    - 특성 데이터와 타겟 데이터를 나누어 줍니다.
    - 특성 행렬은 주로 `X` 로 표현하고 보통 2-차원 행렬입니다(`[n_samples, n_features]`). 주로 NumPy 행렬이나 Pandas 데이터프레임으로 표현합니다.
    - 타겟 배열은 주로 y로 표현하고 보통 1차원 형태(`n_samples`) 입니다. 주로 Numpy 배열이나 Pandas Series로 표현합니다.


- scikit-learn에는 수많은 머신러닝 모델이 구현되어 있는데 모두 유사한 프로세스를 통해서 사용할 수 있습니다.

    - 우선 여러분께서 풀어야 하는 문제를 풀기에 적합한 모델을 선택하여 클래스를 찾아본 후 관련 속성이나 하이퍼파라미터를 확인해 봅니다.
    - 문제에 따라서 약간 차이가 있겠지만 위에서 살펴본 것과 같이 데이터를 준비합니다.
    - `fit()` 메소드를 사용하여 모델을 학습합니다.
    - `predict()` 메소드를 사용하여 새로운 데이터를 예측합니다.

이런 프로세스는 앞으로도 계속 연습할 것이기 때문에 잘 알아 두시기 바랍니다.
- [Basics of the API](https://jakevdp.github.io/PythonDataScienceHandbook/05.02-introducing-scikit-learn.html#Basics-of-the-API)

### 단순선형회귀 (Simple Linear Regression)
- 단순선형회귀는 하나의 독립변수로 종속변수를 설명하는 선형회귀입니다.
- 아래와 같은 식으로 표현할 수 있습니다.

  $ y = {\alpha}x + \beta $

In [ ]:
# scikit-learn에서 Linear Rergression을 가져옵니다.
from sklearn.linear_model import LinearRegression

# 특성과 타겟값을 변수에 저장하고 X 행렬과 y 벡터를 만듭니다.
feature = ['GrLivArea']
target = 'SalePrice'

X = df[feature]
y = df[target]

print(X.shape)
print(y.shape)



In [ ]:
# 모델을 학습합니다.
simple_ols = LinearRegression()
simple_ols.fit(X, y)

새로운 데이터를 예측하는 방법은 두 가지가 있습니다.
1. 보간(interpolate) : 모델이 학습한 데이터 구간 안에 있는 값을 예측하는 방법입니다.
    - `GrLivArea` = 4000인 값을 예측하는 경우입니다.
2. 외삽(extrapolate) : 모델이 학습한 데이터 구간 바깥에 있는 값을 예측하는 방법입니다.
    - `GrLivArea` = 6000인 값을 예측하는 경우입니다.
<br></br>

> 외삽을 했을 때의 문제점은 무엇이 있을까요?

In [ ]:
# GrLivArea가 3500 이상인 데이터를 보겠습니다.
df[df['GrLivArea'] >= 3500 ]

In [ ]:
# 보간
interpolate_x = [[4000]]
interpolate_y = simple_ols.predict(interpolate_x)[0]

# 외삽
extrapolate_x = [[6000]]
extrapolate_y = simple_ols.predict(extrapolate_x)[0]

print(f'GrLivArea가 4000인 경우 예측값은 ${round(interpolate_y,2)}입니다.')
print(f'GrLivArea가 6000인 경우 예측값은 ${round(extrapolate_y,2)}입니다.')

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=df['GrLivArea'], y=df['SalePrice'], alpha=0.6)

plt.scatter(interpolate_x, interpolate_y, color='red')
plt.scatter(extrapolate_x, extrapolate_y, color='green')

plt.text(4000, interpolate_y, 'interpolate value\n of 4000')
plt.text(6000, extrapolate_y, 'extrapolate value\n of 6000')

plt.show()

### 다중선형회귀(Multiple Linear Regression)
- 다중선형회귀는 두 개 이상의 독립변수로 종속변수를 설명하는 선형회귀입니다.
- 아래와 같은 식으로 표현할 수 있습니다.

  $ y = {\alpha_{1}}x_1 + {\alpha_{2}}x_2 + ... + {\alpha_{n}}x_n + \beta $

In [ ]:
# 2개의 특성을 이용하여 다중선형회귀를 만들어보겠습니다.
features = ['GrLivArea', 'OverallQual']
target = 'SalePrice'

X = df[features]
y = df[target]

print(X.shape)
print(y.shape)


In [ ]:
# 3차원 그래프
import numpy as np

fig = plt.figure(figsize=(6,6))
# for 3d plot
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df['GrLivArea'], df['OverallQual'], df['SalePrice'], alpha=0.6)
ax.set_xlabel('GrLivArea', labelpad=12)
ax.set_ylabel('OverallQual', labelpad=10)
ax.set_zlabel('SalePrice', labelpad=20)
plt.suptitle('Housing Prices', fontsize=15)
plt.show()

In [ ]:
# 변수가 3개인 그래프는 hue와 size를 이용해 2차원으로 나타낼 수 있습니다.
plt.figure(figsize=(8,6))
sns.scatterplot(x=df['GrLivArea'], y=df['SalePrice'], hue=df['OverallQual'], size=df['OverallQual'], alpha=0.6)
plt.show()


In [ ]:
# 모델을 학습합니다.
multiple_ols = LinearRegression()
multiple_ols.fit(X, y)

In [ ]:
# 보간
interpolate_x_m = [[4000, 5]]
interpolate_y_m = multiple_ols.predict(interpolate_x_m)[0]

# 외삽
extrapolate_x_m = [[6000, 10]]
extrapolate_y_m = multiple_ols.predict(extrapolate_x_m)[0]

print(f'GrLivArea가 4000이고 OverallQual이 5 경우 예측값은 ${round(interpolate_y_m,2)}입니다.')
print(f'GrLivArea가 6000이고 OverallQual이 10 경우 예측값은 ${round(extrapolate_y_m,2)}입니다.')

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=df['GrLivArea'], y=df['SalePrice'], hue=df['OverallQual'], size=df['OverallQual'], alpha=0.6)
sns.scatterplot(x = [4000, 6000], y=[interpolate_y_m, extrapolate_y_m], color='red')


plt.text(4000, interpolate_y_m, 'interpolate value\n of 4000 & 5')
plt.text(6000, extrapolate_y_m, 'extrapolate value\n of 6000 & 10')

plt.show()

### 다항선형회귀(Polynomial Linear Reregrssion)
- 다항선형회귀는 2차항 이상의 독립변수로 종속변수를 설명하는 선형회귀입니다.
- 독립변수가 2개인 다항선형회귀는 아래와 같은 식으로 나타낼 수 있습니다.(degree=2)

  $ y = {\alpha_{1}}x_1 + {\alpha_{2}}x_2 + {\alpha_{1}}x_1^2 + {\alpha_{2}}x_2^2 + {\alpha_{1}}{x_1}{x_2} + \beta $
<br></br>

> 다항선형회귀도 역시 선형회귀입니다. 왜일까요? 선형회귀에서 말하는 `선형성`이란 무엇일까요?

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

# PolynomialFeatures를 사용하면 다항회귀를 만들 수 있습니다.

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))


features = ['GrLivArea', 'OverallQual']
target = 'SalePrice'

X = df[features]
y = df[target]

# 2차항의 다항선형회귀를 만들어보겠습니다.

poly_ols = PolynomialRegression(degree=2)
poly_ols.fit(X, y)

In [ ]:
# 보간
interpolate_x_p = [[4000, 5]]
interpolate_y_p = poly_ols.predict(interpolate_x_p)[0]

# 외삽
extrapolate_x_p = [[6000, 10]]
extrapolate_y_p = poly_ols.predict(extrapolate_x_p)[0]

print(f'GrLivArea가 4000이고 OverallQual이 5 경우 예측값은 ${round(interpolate_y_p,2)}입니다.')
print(f'GrLivArea가 6000이고 OverallQual이 10 경우 예측값은 ${round(extrapolate_y_p,2)}입니다.')

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=df['GrLivArea'], y=df['SalePrice'], hue=df['OverallQual'], size=df['OverallQual'], alpha=0.6)
sns.scatterplot(x = [4000, 6000], y=[interpolate_y_p, extrapolate_y_p], color='red')


plt.text(4000, interpolate_y_p, 'interpolate value\n of 4000 & 5')
plt.text(6000, extrapolate_y_p, 'extrapolate value\n of 6000 & 10')

plt.show()

## 2.2 회귀 평가지표
- 위에서 기준모델까지 포함하여 총 4가지 회귀모델을 만들어보았습니다.
- 어떤 모델이 가장 예측을 잘 하는 모델일까요?

In [ ]:
comparison = pd.DataFrame(index=[4000, 6000], columns=['Base','Simple', 'Multiple', 'Polynomial'])
comparison['Base'] = [predict, predict]
comparison['Simple'] = [interpolate_y, extrapolate_y]
comparison['Multiple'] = [interpolate_y_m, extrapolate_y_m]
comparison['Polynomial'] = [interpolate_y_p, extrapolate_y_p]
comparison

예측값을 표로 비교해봤을 때는 어떤 모델이 가장 좋은 모델인지 알 수 없습니다.  
모델의 성능을 평가하고 비교할 때는 평가지표를 활용할 수 있습니다.
<br></br>

대표적인 회귀 평가지표는 다음과 같습니다.
* `MSE (Mean Squared Error)` = $\frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat{y_{i}})^{2}$
    - MSE는 제곱오차의 평균입니다.
    - MSE는 미분이 가능하여 비용 함수로 가장 흔하게 사용됩니다.
    - 하지만 제곱을 취하여 이상치에 민감합니다.
<br></br>

* `RMSE (Root Mean Squared Error)` = $\sqrt{MSE}$
    - RMSE는 MSE에 루트를 씌워 상대적으로 덜 민감하게 됩니다.
    - 값의 크기가 실제값과 비슷해지기 때문에 MSE에 비해 이해하기 쉽습니다.
<br></br>

* `MAE (Mean absolute error)` = $\frac{1}{n}\sum_{i=1}^{n}\left | y_{i} - \hat{y_{i}} \right |$
    - MAE는 절대오차의 평균으로 직관적으로 이해하기 쉽습니다.
    - 오차의 절대값이기 때문에 이상치에 민감하지 않습니다.
<br></br>

* `R-squared (Coefficient of determination)` = $1 - \frac{\sum_{i=1}^{n}(y_{i} - \hat{y_{i}})^{2}}{\sum_{i=1}^{n}(y_{i} - \bar{y_{i}})^{2}} = 1 - \frac{SSE}{SST} = \frac {SSR}{SST}$
    - $R^2$ (결정계수)는 모델의 설명력을 나타냅니다.
    - 값이 1에 가까울수록 설명력이 높으며 0에 가까울수록 설명력이 낮습니다.
<br></br>

- 참고
    - SSE(Sum of Squares `Error`, 관측치와 예측치 차이): $\sum_{i=1}^{n}(y_{i} - \hat{y_{i}})^{2}$
    - SSR(Sum of Squares due to `Regression`, 예측치와 평균 차이): $\sum_{i=1}^{n}(\hat{y_{i}} - \bar{y_{i}})^{2}$
    - SST(Sum of Squares `Total`, 관측치와 평균 차이): $\sum_{i=1}^{n}(y_{i} - \bar{y_{i}})^{2}$ , SSE + SSR
<br></br>

> **언제 어떤 평가지표를 활용하는 것이 좋을까요?**

❗️ 회귀 평가지표와 분류 평가지표는 다릅니다. 어떤 문제를 풀고 있는지에 따라 정확한 평가지표를 사용해야합니다!

In [ ]:
# scikit-learn을 통해 손쉽게 평가지표를 사용할 수 있습니다.
# 4가지 모델의 성능을 평가해보겠습니다.

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
# 실제값
y_real = df['SalePrice']

# 기준 모델
y_base = [predict]*len(df)

# 단순선형회귀
feature = ['GrLivArea']
X_simple = df[feature]
y_simple = simple_ols.predict(X_simple)

# 다중선형회귀
features = ['GrLivArea', 'OverallQual']
X_multiple = df[features]
y_multiple = multiple_ols.predict(X_multiple)

# 다항선형회귀
y_poly = poly_ols.predict(X_multiple)

In [ ]:
# mse, rmse, mae, r2를 출력하는 함수입니다.

def eval_models(y_pred, y_real=y_real) :
    mse = mean_squared_error(y_real, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_real, y_pred)
    r2 = r2_score(y_real, y_pred)

    return mse, rmse, mae, r2

In [ ]:
base_mse, base_rmse, base_mae, base_r2 = eval_models(y_base)
simple_mse, simple_rmse, simple_mae, simple_r2 = eval_models(y_simple)
multiple_mse, multiple_rmse, multiple_mae, multiple_r2 = eval_models(y_multiple)
poly_mse, poly_rmse, poly_mae, poly_r2 = eval_models(y_poly)

In [ ]:
comparison_metrics = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Base','Simple', 'Multiple', 'Polynomial'])
comparison_metrics['Base'] = [base_mse, base_rmse, base_mae, base_r2 ]
comparison_metrics['Simple'] = [simple_mse, simple_rmse, simple_mae, simple_r2]
comparison_metrics['Multiple'] = [multiple_mse, multiple_rmse, multiple_mae, multiple_r2]
comparison_metrics['Polynomial'] = [poly_mse, poly_rmse, poly_mae, poly_r2]
comparison_metrics

평가지표로 4가지 모델을 비교해본 결과
- 기준모델(Base)의 에러가 가장 크며 결정계수가 0인 것을 봤을 때 전혀 학습이 되지 않은 모델이라고 할 수 있습니다.
- 다항회귀모델(2차항)은 다른 모델에 비해 에러가 가장 작고 결정계수가 0.8로 가장 크므로 가장 좋은 모델이라고 할 수 있습니다.
- 다중선형회귀모델은 다항선형회귀모델에 비해서는 에러가 조금 나지만 단순선형회귀모델과 비교했을 때는 많이 개선된 모델이라고 할 수 있습니다.
<br></br>

이처럼 평가지표를 사용해 모델을 비교하고 개선할 수 있습니다.

## 2.3 선형회귀모델의 계수(Coefficients)

- 선형회귀모델의 가장 큰 장점은 직관적으로 모델을 해석할 수 있다는 것입니다.
- 위에서 만든 모델은 독립변수와 종속변수(주택 판매 가격) 사이에 어떤 규칙을 학습했을까요?
- 이 질문에 대한 답을 하기 위해 `LinearRegression` 객체의 `coef_`, `intercept_` 속성을 확인해 보겠습니다.
    - [sckit-learn.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)


In [ ]:
# 다중선형회귀를 예시로 들어보겠습니다.
# 회귀계수
print('coefficient : ', multiple_ols.coef_)

## 절편(intercept)
print('Intercept : ', multiple_ols.intercept_)

In [ ]:
# 위의 결과를 활용해 다중선형회귀식을 구할 수 있습니다.
print(f'다중선형회귀식 : y = GrLivArea * {multiple_ols.coef_[0]:.3f} + OverallQual * {multiple_ols.coef_[1]:.3f} {multiple_ols.intercept_:.3f}')

회귀계수는 어떻게 해석해야할까요?
- GrLivArea가 1sqft 증가할 때마다 주택 판매 가격은 약 55.86 달러 증가합니다.
- OverallQual이 1단계 증가할 때마다 주택 판매 가격은 약 32849.04 달러 증가합니다.
- 절편은 해석하지 않습니다.
<br></br>

> OverallQual의 회귀계수가 GrLivArea보다 현저히 큽니다. 이 경우 OverallQual의 영향력이 GrLivArea보다 크다고 말할 수 있을까요?

#### 단순선형회귀모델로 예측함수를 만들어 새로운 데이터를 반복해서 예측하고 coefficient의 영향을 설명해 봅시다:

In [ ]:
def explain_prediction(sqft):
    y_pred =simple_ols.predict([[sqft]])
    pred = f"{int(sqft)} sqft 주택 가격 예측: ${int(y_pred[0])} (1 sqft당 추가금: ${int(simple_ols.coef_[0])})"

    return pred

In [ ]:
## square_feet = 4000 인 테스트 데이터로 예측
print(explain_prediction(4000))

#### ipywidgets를 사용해서 sqft를 바꾸어 가며 예측해 보겠습니다.

In [ ]:
from ipywidgets import interact

# 데코레이터 interact를 추가합니다.
@interact
def explain_prediction(sqft=(500,10000)):
    y_pred = simple_ols.predict([[sqft]])
    pred = f"{int(sqft)} sqft 주택 가격 예측: ${int(y_pred[0])} (1 sqft당 추가금: ${int(simple_ols.coef_[0])})"

    return pred

## 참고자료

#### 머신러닝

- [Introduction to Machine Learning](https://developers.google.com/machine-learning/intro-to-ml/what-is-ml)
- [Deep Learning with Python](https://livebook.manning.com/book/deep-learning-with-python/chapter-1/16)

#### 지도학습 vs 비지도학습
- [Supervised vs. Unsupervised Machine Learning: What's the Difference?](https://youtu.be/rHeaoaiBM6Y)

#### Scikit-Learn
- [Python Data Science Handbook, Chapter 5.2: Introducing Scikit-Learn](https://jakevdp.github.io/PythonDataScienceHandbook/05.02-introducing-scikit-learn.html#Basics-of-the-API)
- [2.4.2.2. Supervised Learning](https://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/tutorial/text_analytics/general_concepts.html#supervised-learning-model-fit-x-y)
- [sklearn.linear_model.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- [sklearn.metrics.mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html)


#### 최소제곱법
- [Ordinary Least-Squares Regression](https://datajobs.com/data-science-repo/OLS-Regression-[GD-Hutcheson].pdf)
- [수학산책-최소제곱법](https://terms.naver.com/entry.nhn?cid=58944&docId=3569970&categoryId=58970)

#### (참고) 더 세련된 시각화툴: Plotly
- [Plotly Express](https://plot.ly/python/plotly-express/)
- [plotly_express.scatter](https://www.plotly.express/plotly_express/#plotly_express.scatter)

#### ipywidgets interact
- [Using Interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html#Using-Interact)